In [2]:
%load_ext sql
%sql sqlite://

## Продолжаем работу с таблицами

In [8]:
%%sql

DROP TABLE IF EXISTS Employees;

--PRAGMA foreign_keys = ON;

CREATE TABLE Employees(
    id int PRIMARY KEY,
    name varchar(20) NOT NULL,
    phone varchar(20),
    title varchar(20) NOT NULL,
    manager_id integer,
    FOREIGN KEY (manager_id) REFERENCES Employees(id)
);

 * sqlite://
Done.
Done.


[]

In [4]:
%%sql

--INSERT INTO Employees values  (1, 'Vasyugin', '+111', 'Director', NULL);
INSERT INTO Employees values  (2, 'Jones', '+112', 'Vice President', 1);
INSERT INTO Employees values  (3, 'Smith', '+113', 'Vice President', 1);
INSERT INTO Employees values (4, 'Velaskes', '+114', 'Developer', 2);
INSERT INTO Employees values (5, 'Simon', '+115', 'Analyst', 2);
INSERT INTO Employees values (6, 'Johnson', '+116', 'Engineer', 3);
INSERT INTO Employees values (7, 'Lee', '+117', 'Engineer', 3);

 * sqlite://
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [7]:
%%sql

PRAGMA foreign_key_check; -- проверка целостности

 * sqlite://
Done.


table,rowid,parent,fkid
Employees,1,Employees,0
Employees,2,Employees,0


In [6]:
%%sql

PRAGMA foreign_keys = ON;

 * sqlite://
Done.


[]

In [9]:
%%sql
DROP TABLE IF EXISTS Students;
DROP TABLE IF EXISTS Faculties;

CREATE TABLE Faculties(
    id integer PRIMARY KEY,
    name varchar(50) NOT NULL,
    UNIQUE (name) -- уникальные значения
);

CREATE TABLE Students(
    id integer PRIMARY KEY,
    name varchar(50) NOT NULL,
    gpa decimal(2,2),
    faculty_id integer,
    FOREIGN KEY (faculty_id) REFERENCES Faculties(id)
);

 * sqlite://
Done.
Done.
Done.
Done.


[]

In [10]:
%%sql
INSERT INTO Faculties values (1, 'IT');
INSERT INTO Faculties values (2, 'KIB');

INSERT INTO Students values (1, 'Sidorov', 3.0, 1);
INSERT INTO Students values (2, 'Petrov', 3.5, 2);
INSERT INTO Students values (3, 'Verikov', 4.0, 1);
INSERT INTO Students values (4, 'Smith', 4.2, 2);
INSERT INTO Students values (5, 'Lee', 4.5, 1);
INSERT INTO Students values (6, 'Vorontsov', 3.2, NULL);

INSERT INTO Employees values  (1, 'Vasyugin', '+111', 'Director', NULL);
INSERT INTO Employees values  (2, 'Jones', '+112', 'Vice President', 1);
INSERT INTO Employees values  (3, 'Smith', '+113', 'Vice President', 1);
INSERT INTO Employees values (4, 'Velaskes', '+114', 'Developer', 2);
INSERT INTO Employees values (5, 'Simon', '+115', 'Analyst', 2);
INSERT INTO Employees values (6, 'Johnson', '+116', 'Engineer', 3);
INSERT INTO Employees values (7, 'Lee', '+117', 'Engineer', 3);

 * sqlite://
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

## Оператор WITH

In [11]:
%%sql
WITH g AS
(
SELECT faculty_id, min(gpa) as min_gpa
    FROM Students
 GROUP
    BY faculty_id
)
SELECT s.*
    FROM Students s
INNER JOIN g
    ON s.faculty_id = g.faculty_id
WHERE s.gpa > g.min_gpa;

 * sqlite://
Done.


id,name,gpa,faculty_id
3,Verikov,4,1
4,Smith,4.2,2
5,Lee,4.5,1


In [12]:
%%sql
SELECT * FROM g;

 * sqlite://
(sqlite3.OperationalError) no such table: g
[SQL: SELECT * FROM g;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [13]:
%%sql
WITH KIB_fac_id AS
(
SELECT id
    FROM Faculties
WHERE name = 'KIB'
),
min_gpa as
(
SELECT MIN(gpa) as min_gpa
    FROM Students
WHERE faculty_id = (SELECT id FROM KIB_fac_id)
)
SELECT *
    FROM Students
WHERE gpa > (SELECT min_gpa from min_gpa);

 * sqlite://
Done.


id,name,gpa,faculty_id
3,Verikov,4,1
4,Smith,4.2,2
5,Lee,4.5,1


## Представления VIEW

In [14]:
%%sql
CREATE VIEW IF NOT EXISTS StudentsAndFaculties AS
SELECT s.*, f.name as faculty_name
    FROM Students s
LEFT OUTER JOIN Faculties F
    ON s.faculty_id = f.id;

 * sqlite://
Done.


[]

Представления — виртуальные таблицы, которые содержат запросы, динамически извлекающие используемые данные.

In [15]:
%%sql
SELECT * FROM StudentsAndFaculties;

 * sqlite://
Done.


id,name,gpa,faculty_id,faculty_name
1,Sidorov,3,1,IT
2,Petrov,3.5,2,KIB
3,Verikov,4,1,IT
4,Smith,4.2,2,KIB
5,Lee,4.5,1,IT
6,Vorontsov,3.2,None,None


In [16]:
%%sql
DROP TABLE Students;

 * sqlite://
Done.


[]

In [17]:
%%sql
SELECT * FROM StudentsAndFaculties;

 * sqlite://
(sqlite3.OperationalError) no such table: main.Students
[SQL: SELECT * FROM StudentsAndFaculties;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


## CASE WHEN

In [18]:
%%sql
SELECT *,
    CASE WHEN gpa > 4.0
              THEN 'A'
            WHEN gpa > 3.0
              THEN 'B'
            ELSE 'C'
    END as mark
    FROM Students;

 * sqlite://
(sqlite3.OperationalError) no such table: Students
[SQL: SELECT *,
    CASE WHEN gpa > 4.0
              THEN 'A'
            WHEN gpa > 3.0
              THEN 'B'
            ELSE 'C'
    END as mark
    FROM Students;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [19]:
%%sql
SELECT CASE WHEN gpa > 4.0
              THEN 'A'
            WHEN gpa > 3.0
              THEN 'B'
            ELSE 'C'
       END as mark,
       count(*) as cnt
    FROM Students
GROUP BY CASE WHEN gpa > 4.0
              THEN 'A'
            WHEN gpa > 3.0
              THEN 'B'
            ELSE 'C'
       END;

 * sqlite://
(sqlite3.OperationalError) no such table: Students
[SQL: SELECT CASE WHEN gpa > 4.0
              THEN 'A'
            WHEN gpa > 3.0
              THEN 'B'
            ELSE 'C'
       END as mark,
       count(*) as cnt
    FROM Students
GROUP BY CASE WHEN gpa > 4.0
              THEN 'A'
            WHEN gpa > 3.0
              THEN 'B'
            ELSE 'C'
       END;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [20]:
%%sql
SELECT mark, count(*) FROM
    (SELECT CASE WHEN gpa > 4.0
                  THEN 'A'
                WHEN gpa > 3.0
                  THEN 'B'
                ELSE 'C'
          END as mark
        FROM Students) as c
GROUP BY c.mark;

 * sqlite://
(sqlite3.OperationalError) no such table: Students
[SQL: SELECT mark, count(*) FROM
    (SELECT CASE WHEN gpa > 4.0
                  THEN 'A'
                WHEN gpa > 3.0
                  THEN 'B'
                ELSE 'C'
          END as mark
        FROM Students) as c
GROUP BY c.mark;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


## Рекурсивные запросы

In [21]:
%%sql
WITH RECURSIVE cnt(x) AS (
    VALUES(1)

    UNION ALL

    SELECT x+1 FROM cnt
    WHERE x < 10
    )
SELECT x FROM cnt;

 * sqlite://
Done.


x
1
2
3
4
5
6
7
8
9
10


In [23]:
%%sql
select * from Employees

 * sqlite://
Done.


id,name,phone,title,manager_id
1,Vasyugin,+111,Director,None
2,Jones,+112,Vice President,1
3,Smith,+113,Vice President,1
4,Velaskes,+114,Developer,2
5,Simon,+115,Analyst,2
6,Johnson,+116,Engineer,3
7,Lee,+117,Engineer,3


In [26]:
%%sql
WITH recursive ManagerPath(title_path, manager_id) AS (
    SELECT title as title_path, id as manager_id
        FROM Employees
    WHERE manager_id is null

    UNION

    SELECT ManagerPath.title_path || '->' || Employees.title as title_path, id as manager_id
        FROM ManagerPath
    INNER JOIN Employees
    ON ManagerPath.manager_id = Employees.manager_id
    )
select *
from ManagerPath

 * sqlite://
Done.


title_path,manager_id
Director,1
Director->Vice President,2
Director->Vice President,3
Director->Vice President->Developer,4
Director->Vice President->Analyst,5
Director->Vice President->Engineer,6
Director->Vice President->Engineer,7


In [27]:
%%sql
WITH recursive ManagerPath(title_path, manager_id, level) AS (
    SELECT title as title_path, id as manager_id, 1 as level
        FROM Employees
    WHERE manager_id is null

    UNION

    SELECT ManagerPath.title_path || '->' || Employees.title as title_path,
          id as manager_id,
          ManagerPath.level + 1 as level
    FROM ManagerPath
    INNER JOIN Employees using (manager_id)
)
select * from ManagerPath

 * sqlite://
Done.


title_path,manager_id,level
Director,1,1
Director->Vice President,2,2
Director->Vice President,3,2
Director->Vice President->Developer,4,3
Director->Vice President->Analyst,5,3
Director->Vice President->Engineer,6,3
Director->Vice President->Engineer,7,3


In [ ]:
%%sql
WITH RECURSIVE cnt(x) AS (
    VALUES(1)

    UNION ALL

    SELECT x+1 FROM cnt
    WHERE x > 0  -- бесконечная рекурсия (работать не будет)
    )
SELECT x FROM cnt;

 * sqlite://
Done.


## Ограничение целостности CHECK

In [28]:
%%sql
DROP TABLE IF EXISTS Clients;

CREATE TABLE Clients(
    ID integer PRIMARY KEY,
    FirstName varchar(255) NOT NULL,
    LastName varchar(255) NOT NULL,
    Age integer,
    CHECK (Age>=18)
);

 * sqlite://
Done.
Done.


[]

In [29]:
%%sql
INSERT INTO Clients (FirstName, LastName, Age) VALUES ("Антон", "Иванов", 23);
INSERT INTO Clients (FirstName, LastName, Age) VALUES ("Андрей", "Степанов", 27);
INSERT INTO Clients (FirstName, LastName, Age) VALUES ("Наталья", "Ештукова", 20);
INSERT INTO Clients (FirstName, LastName, Age) VALUES ("Виктория", "Андреева", 37);
INSERT INTO Clients (FirstName, LastName, Age) VALUES ("Илья", "Ильин", 47);

 * sqlite://
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [30]:
%%sql
SELECT * FROM Clients;

 * sqlite://
Done.


ID,FirstName,LastName,Age
1,Антон,Иванов,23
2,Андрей,Степанов,27
3,Наталья,Ештукова,20
4,Виктория,Андреева,37
5,Илья,Ильин,47


In [31]:
%%sql
INSERT INTO Clients (FirstName, LastName, Age) VALUES ("Иван", "Воронцов", 17);

 * sqlite://
(sqlite3.IntegrityError) CHECK constraint failed: Age>=18
[SQL: INSERT INTO Clients (FirstName, LastName, Age) VALUES ("Иван", "Воронцов", 17);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [32]:
%%sql
INSERT INTO Clients (FirstName, LastName) VALUES ("Иван", "Воронцов");

 * sqlite://
1 rows affected.


[]

In [33]:
%%sql
SELECT * FROM Clients;

 * sqlite://
Done.


ID,FirstName,LastName,Age
1,Антон,Иванов,23
2,Андрей,Степанов,27
3,Наталья,Ештукова,20
4,Виктория,Андреева,37
5,Илья,Ильин,47
6,Иван,Воронцов,None


## Математические функции

In [34]:
%%sql
DROP TABLE IF EXISTS Numbers;

CREATE TABLE Numbers(
    a integer,
    b integer
);

 * sqlite://
Done.
Done.


[]

In [35]:
%%sql
INSERT INTO Numbers VALUES (random(), 0); --между -9223372036854775808 и +9223372036854775807
INSERT INTO Numbers VALUES (random(), 0);
INSERT INTO Numbers VALUES (random(), 0);
SELECT * FROM Numbers;

 * sqlite://
1 rows affected.
1 rows affected.
1 rows affected.
Done.


a,b
1978763835261631773,0
-3930439263297638603,0
-3260380857469159290,0


In [36]:
%%sql
INSERT INTO Numbers VALUES (abs(random() % 10), abs(random() % 10)); --между 0 and 10
INSERT INTO Numbers VALUES ((abs(random()) % (10-1)+1), (abs(random()) % (10-1)+1)); --между 1 and 10
INSERT INTO Numbers VALUES (NULL, NULL);
SELECT * FROM Numbers;

 * sqlite://
1 rows affected.
1 rows affected.
1 rows affected.
Done.


a,b
1978763835261631773,0
-3930439263297638603,0
-3260380857469159290,0
9,9
7,5
None,None


In [37]:
%%sql
SELECT a, b, sin(a), cos(a), atan(a), sqrt(a), ln(a), pow(a,b) FROM Numbers;

 * sqlite://
Done.


a,b,sin(a),cos(a),atan(a),sqrt(a),ln(a),"pow(a,b)"
1978763835261631773,0,0.8015578249131222,-0.5979172629390662,1.5707963267948966,1406685407.3536243,42.12900399800143,1.0
-3930439263297638603,0,-0.5539924542184859,-0.8325216878069777,-1.5707963267948966,None,None,1.0
-3260380857469159290,0,-0.10162688640189642,0.9948225851679565,-1.5707963267948966,None,None,1.0
9,9,0.4121184852417566,-0.9111302618846769,1.460139105621001,3.0,2.1972245773362196,387420489.0
7,5,0.6569865987187891,0.7539022543433046,1.4288992721907328,2.6457513110645907,1.9459101490553132,16807.0
None,None,None,None,None,None,None,None
